In [1]:
import pandas as pd
import re
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator


<p style="border:3px #ff0000  solid;">
Задание: Спрогнозировать сердечную недостаточность

<p style="border:3px #ff0000  solid;">
1. Загрузим треировочную и тестовую выборки. Проверим и устраним, при необходимости, пропуски, а также проведем другие подготовительные действия

In [2]:
heart_train_df = pd.read_csv('C:/Users/Z/PycharmProjects/data_science_1t/task_2.8/heart_adapt_train.csv')
heart_test_df = pd.read_csv('C:/Users/Z/PycharmProjects/data_science_1t/task_2.8/heart_adapt_test.csv')

     1.1 Оберните весь конвейер преобразований в Pipeline

In [3]:
class MyTransformer(TransformerMixin, BaseEstimator):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = pd.get_dummies(X, drop_first=True)
        #X.clolumns = [re.sub('\\.+', '_', i).lower() for i in X.columns]    #Почему-то не работает
        X = self.convert_bool(X)
        X = X.fillna(X.mean())
        X = X.reset_index(drop=True)      
        return X
    
    def convert_bool(self, df):
        bool_column = df.select_dtypes(include=bool).columns
        for column in bool_column:
            df[column] = df[column].astype(int)
        return df

In [4]:
#Создаем трансформер и преобразуем данные
myTransformer = MyTransformer()
myTransformer.fit(heart_train_df)
heart_train_df = myTransformer.transform(heart_train_df)
heart_test_df = myTransformer.transform(heart_test_df)


In [5]:
heart_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589 entries, 0 to 588
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                589 non-null    float64
 1   RestingBP          589 non-null    float64
 2   Cholesterol        589 non-null    float64
 3   FastingBS          589 non-null    int64  
 4   MaxHR              589 non-null    int64  
 5   Oldpeak            589 non-null    float64
 6   HeartDisease       589 non-null    int64  
 7   Sex_M              589 non-null    int32  
 8   ChestPainType_ATA  589 non-null    int32  
 9   ChestPainType_NAP  589 non-null    int32  
 10  ChestPainType_TA   589 non-null    int32  
 11  RestingECG_Normal  589 non-null    int32  
 12  RestingECG_ST      589 non-null    int32  
 13  ExerciseAngina_Y   589 non-null    int32  
 14  ST_Slope_Flat      589 non-null    int32  
 15  ST_Slope_Up        589 non-null    int32  
dtypes: float64(4), int32(9), i

In [6]:
heart_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                197 non-null    float64
 1   RestingBP          197 non-null    float64
 2   Cholesterol        197 non-null    float64
 3   FastingBS          197 non-null    int64  
 4   MaxHR              197 non-null    int64  
 5   Oldpeak            197 non-null    float64
 6   HeartDisease       197 non-null    int64  
 7   Sex_M              197 non-null    int32  
 8   ChestPainType_ATA  197 non-null    int32  
 9   ChestPainType_NAP  197 non-null    int32  
 10  ChestPainType_TA   197 non-null    int32  
 11  RestingECG_Normal  197 non-null    int32  
 12  RestingECG_ST      197 non-null    int32  
 13  ExerciseAngina_Y   197 non-null    int32  
 14  ST_Slope_Flat      197 non-null    int32  
 15  ST_Slope_Up        197 non-null    int32  
dtypes: float64(4), int32(9), i

In [7]:
heart_train_df.HeartDisease.value_counts()

HeartDisease
1    381
0    208
Name: count, dtype: int64

<p style="border:3px #00B344  solid;">
Мы избавились от пропусков. Данные подготовлены к дальнейшей работе. В целевом признаке имеется дисбаланс классов, который необходимо учесть при обучении моделей.
</p>

<p style="border:3px #ff0000  solid;">
2. Подберите оптимальный вариант прогнозной модели с помощью GridSearchCV

    Отделяем признаки

In [8]:
train_target = heart_train_df['HeartDisease']
train_features = heart_train_df.drop(['HeartDisease'], axis=1)
test_target = heart_test_df['HeartDisease']
test_features = heart_test_df.drop(['HeartDisease'], axis=1)

    Подготовим Pipeline

In [9]:
pipe = Pipeline([  
  ('ohe_types', MyTransformer()),
  ('scaler', StandardScaler()),
  ('classify', DecisionTreeClassifier(class_weight='balanced', random_state=0))
])

    Подберем оптимальный вариант прогнозной модели с помощью GridSearchCV

In [10]:
params = [
    {'classify': [LogisticRegression(class_weight='balanced', random_state=0)]}, 
    {'classify': [DecisionTreeClassifier(class_weight='balanced', random_state=0)], 'classify__max_depth': [2, 5, 10, 20]},
    {'classify': [RandomForestClassifier(class_weight='balanced', random_state=0)], 'classify__max_depth': [2, 5, 10, 20], 'classify__n_estimators' : range(10, 51, 10)}
]

grid_search = GridSearchCV(pipe, param_grid=params, cv=5, scoring='roc_auc')
grid_search.fit(X=train_features, y=train_target)
print(
    'Качество модели на тестовой выборке c лучшей моделью:', 
    {roc_auc_score(test_target, grid_search.predict_proba(test_features)[:, 1])}
)

Качество модели на тестовой выборке c лучшей моделью: {0.9081552305961754}


In [11]:
grid_search.best_estimator_

Pipeline(steps=[('ohe_types', MyTransformer()), ('scaler', StandardScaler()),
                ('classify',
                 RandomForestClassifier(class_weight='balanced', max_depth=20,
                                        n_estimators=50, random_state=0))])

<p style="border:3px #00B344  solid;">
Выводы:

    1. В результате проведенной работы закреплены знания в оборачивании конвейера преобразований в Pipeline.
    2. Входе исследования обучено три модели, из которых самой эффективной оказалась RandomForestClassifier с параметрами max_depth=20 и n_estimators=50.
    3. Качество модели на тестовой выборке составило 0.91.
    4. Так как в целевом признаке имеется дисбаланс классов, при обучении моделей применялся параметр class_weight='balanced'.